In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Preparation of Data and Exploratory Data Analysis

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from shutil import copyfile
from os import getcwd

In [ ]:
!unzip -o ../input/dogs-vs-cats-redux-kernels-edition/train.zip

In [ ]:
ls -l -t /kaggle/working/train | head -1

In [ ]:
!unzip -o ../input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
#ls /kaggle/working/test

In [ ]:
ls -l -t /kaggle/working/test | head -1

In [ ]:
cd /kaggle/working/train

In [ ]:
mkdir cat dog

In [ ]:
mv  ./dog.* ./dog

In [ ]:
mv ./cat.* ./cat

In [ ]:
cd /kaggle/working/test

In [ ]:
ls -l -t /kaggle/working/test | head -3

In [ ]:
image_size = (180,180)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.3, horizontal_flip=1)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_ds = train_datagen.flow_from_directory('/kaggle/working/train', target_size = image_size, subset = 'training', batch_size = 32, class_mode = 'binary', shuffle =False)
val_ds = train_datagen.flow_from_directory('/kaggle/working/train', target_size = image_size, subset = 'validation', batch_size = 32, class_mode = 'binary', shuffle =1)

In [ ]:
print(train_ds[0])

In [ ]:
# from sklearn.model_selection import train_test_split
# train,validation= train_test_split(df, test_size=0.1)
# train = train.reset_index(drop=True)
# validation = validation.reset_index(drop=True)

In [ ]:
# train_ds = train_ds.prefetch(buffer_size=32)
# val_ds = val_ds.prefetch(buffer_size=32)

In [ ]:
# model = sequential()

# model.add()

In [ ]:
from keras.applications import InceptionResNetV2

model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))

In [ ]:
model.summary()

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
my_base_model = keras.applications.DenseNet201(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(100, 100, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
my_base_model.trainable = False

# Create new model on top
my_inputs = keras.Input(shape=(100, 100, 3))
# x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
# scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
# x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = my_base_model(my_inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
# x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
# x = keras.layers.Dense(128, activation='relu')(x)
# x = keras.layers.Dense(128, activation='relu')(x)
my_outputs = keras.layers.Dense(1, activation='sigmoid')(x)
my_model = keras.Model(my_inputs,my_outputs)

my_model.summary()

In [ ]:
len(train_ds.filenames)

In [ ]:
import tensorflow.keras as keras
my_model.compile( optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy'])
early_stopping  = keras.callbacks.EarlyStopping(min_delta=0.0005, patience=5)
epochs = 30
my_model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[early_stopping])

In [ ]:
#pip install tf-nightly
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
cd /kaggle/working/

In [ ]:
mkdir testf

In [ ]:
cd /kaggle/working/testf

In [ ]:
#rmdir test2

In [ ]:
mkdir testf2

In [ ]:
mv /kaggle/working/test /kaggle/working/testf/testf2

In [ ]:
cd /kaggle/working

In [ ]:
mkdir testn

In [ ]:
cp /kaggle/working/train/dog/

In [ ]:
# path = '/kaggle/working/testf'

# test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   path,
#   seed=42,
#   image_size=(100, 100),
#   batch_size=32,
#   interpolation='nearest',
#   shuffle=False)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_ds = test_datagen.flow_from_directory('/kaggle/working/testf', 
                                           target_size = image_size,
                                           class_mode = 'binary',
                                          shuffle = False)

In [ ]:
test_ds.filenames[1]

In [ ]:
test_ds

In [ ]:
test_filenames = os.listdir("/kaggle/working/testf/testf2/test")
test_df = pd.DataFrame({
    'filename': test_filenames
})

In [ ]:
test_df['files'] = test_df['filename'].apply(lambda x: x.replace('.jpg', ''))
test_df['files'] = test_df['files']
test_df2 = test_df.sort_values('files')
test_df2

In [ ]:
test_ds.filenames[2]

In [ ]:
#test_ds.reset()
#pred= my_model.predict(test_ds)
pred = my_model.predict(test_ds)
# index = pd.Series([x.split('.')[0] for x in test_df2.filename])

#predicted_class_indices=[1 if x >= 0.5 else 0 for x in pred]

# labels=(test_ds.class_indices)
# labels2=dict((v,k) for k,v in labels.items())
# predictions=[labels2[k] for k in predicted_class_indices]

# print(predictions)

In [ ]:
new_pred = pred.reshape(-1)

In [ ]:
index = pd.Series([x.split('.')[0] for x in test_df2.filename])
index

In [ ]:
predic = pd.DataFrame({'id': index, 'label': new_pred})

In [ ]:
predic[predic['id'] == '4']

In [ ]:
ls /kaggle/working/testf/testf2/test

In [ ]:
print(predic['label'])

In [ ]:
print(index)

In [ ]:
# print(pred)
# pred = [ item for elem in pred for item in elem]
# print(pred)

In [ ]:
new_pred = new_pred.clip(min = 0.002, max = 0.998)

In [ ]:
#counter = range(1, len(pred) + 1)

solution = pd.DataFrame({"id": index, "label":new_pred})
sample_submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
#sample_submission.tail()
solution.to_csv("/kaggle/working/dogsVScats.csv", index = False)

In [ ]:
# sample_submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
# sample_submission.tail()

In [ ]:
pip install -U notebook-as-pdf

In [ ]:
jupyter-nbconvert --to HTML MSBA.Section2.Anurag.ipynb

## 2. Building a Small Model from Scratch

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3,activation="relu", input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=64, kernel_size=3,activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callback=EarlyStopping(monitor="val_loss", patience=2)
callback_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=2, factor=0.5, min_lr=0.00001)

In [ ]:

history=model.fit(train_ds, validation_data=val_ds, epochs=12, callbacks=[early_stopping])

## 4. Preparing Test Data and Submission

In [ ]:
# Extracting the zip file

test_zip = '/kaggle/working/test/'
zip_ref = zipfile.ZipFile(test_zip, 'r')
zip_ref.extractall('/kaggle/working/test/')
zip_ref.close()

In [ ]:
test_dir = '/kaggle/working/test/'
test_images = os.listdir(os.path.join(test_dir))
test_images[:10]

In [ ]:
test_df = pd.DataFrame({'Image': test_images})
test_df.head()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  directory="/kaggle/working/test",
                                                 x_col="Image",
                                                 y_col=None,
                                                  class_mode  = None,
                                                 target_size=(150,150),
                                                shuffle = True,
                                                batch_size=20)

In [ ]:
predictions = model.predict(test_generator,steps = np.ceil(12500/20))
predictions

In [ ]:
test_df["category"]=pd.DataFrame(predictions, columns=["category"])
test_df

In [ ]:
def labelizor(prediction):
    if prediction > 0.5:
        return 1
    else:
        return 0

In [ ]:
test_df["category"] = test_df["category"].apply(labelizor)
test_df

In [ ]:
plt.figure(figsize=(13,10))
sns.countplot(data=test_df, x="category",palette="magma")

In [ ]:
test_df=test_df.reset_index()
test_df

In [ ]:
test_df=test_df.rename(columns={"index": "id"})
test_df

In [ ]:
submission_df=test_df.copy()
submission_df.drop("Image", axis=1, inplace=True)
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)